<a href="https://colab.research.google.com/github/anwarghammam/Anwar-s-Resume/blob/main/Assignment2_Anwar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset [Amazon Reviews Sentiment Analysis](https://www.kaggle.com/code/sapnasaw/sentimentl-analysis-of-amazonreviews/input)

## Mount colab to Drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [13]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder

# Download necessary nltk resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#Download Dataset

## Read Data (It was first dowloaded from kaggle and imported to my drive)


The data from the source was already splited, I compbined them and then later splited them.

In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP/amazonreviews.csv',delimiter='\t', encoding='utf-8')


In [5]:
df

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
9995,pos,A revelation of life in small town America in ...
9996,pos,Great biography of a very interesting journali...
9997,neg,Interesting Subject; Poor Presentation: You'd ...
9998,neg,Don't buy: The box looked used and it is obvio...


##Tokenization


In [7]:
df['tokens'] = df['review'].apply(nltk.word_tokenize)


In [8]:
df['tokens']

,tokens
0,"[Stuning, even, for, the, non-gamer, :, This, ..."
1,"[The, best, soundtrack, ever, to, anything, .,..."
2,"[Amazing, !, :, This, soundtrack, is, my, favo..."
3,"[Excellent, Soundtrack, :, I, truly, like, thi..."
4,"[Remember, ,, Pull, Your, Jaw, Off, The, Floor..."
...,...
9995,"[A, revelation, of, life, in, small, town, Ame..."
9996,"[Great, biography, of, a, very, interesting, j..."
9997,"[Interesting, Subject, ;, Poor, Presentation, ..."
9998,"[Do, n't, buy, :, The, box, looked, used, and,..."


## using the lemmatization process.

In [9]:
# Helper function to convert nltk POS tags to wordnet POS tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

# Lemmatize tokens with their POS tags
def lemmatize_with_pos(tokens):
    pos_tagged = nltk.pos_tag(tokens)  # Perform POS tagging on the full token list
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tagged]
    return lemmatized_tokens

lemmatizer = WordNetLemmatizer()

# Apply the function to lemmatize tokens with correct POS
df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_with_pos)

In [10]:
df['lemmatized_tokens']

,lemmatized_tokens
0,"[Stuning, even, for, the, non-gamer, :, This, ..."
1,"[The, best, soundtrack, ever, to, anything, .,..."
2,"[Amazing, !, :, This, soundtrack, be, my, favo..."
3,"[Excellent, Soundtrack, :, I, truly, like, thi..."
4,"[Remember, ,, Pull, Your, Jaw, Off, The, Floor..."
...,...
9995,"[A, revelation, of, life, in, small, town, Ame..."
9996,"[Great, biography, of, a, very, interesting, j..."
9997,"[Interesting, Subject, ;, Poor, Presentation, ..."
9998,"[Do, n't, buy, :, The, box, look, use, and, it..."


## One Hot Encoding

In [11]:
# Flatten lemmatized tokens and one-hot encode them
unique_tokens = set(token for tokens in df['lemmatized_tokens'] for token in tokens)
encoder = OneHotEncoder(sparse_output=False)
encoded_tokens = encoder.fit_transform([[token] for token in unique_tokens])

# Map tokens to their one-hot encoded vectors
one_hot_dict = dict(zip(unique_tokens, encoded_tokens))
print("Sample One-Hot Encoded Dictionary:", list(one_hot_dict.items())[:10])

Sample One-Hot Encoded Dictionary: [('seriously.Eoghan', array([0., 0., 0., ..., 0., 0., 0.])), ('Fiction', array([0., 0., 0., ..., 0., 0., 0.])), ('amazingly', array([0., 0., 0., ..., 0., 0., 0.])), ('Deucalion', array([0., 0., 0., ..., 0., 0., 0.])), ('slept', array([0., 0., 0., ..., 0., 0., 0.])), ('Lagwagon', array([0., 0., 0., ..., 0., 0., 0.])), ('re-released', array([0., 0., 0., ..., 0., 0., 0.])), ('JAPANESE', array([0., 0., 0., ..., 0., 0., 0.])), ('Speelbound', array([0., 0., 0., ..., 0., 0., 0.])), ('Dreadful', array([0., 0., 0., ..., 0., 0., 0.]))]


In [12]:
# Prepare sentences for Word2Vec
lemmatized_sentences = df['lemmatized_tokens'].tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=lemmatized_sentences, vector_size=100, window=5, min_count=1, sg=1)

# Print Word2Vec vectors for 10 words
for word in list(word2vec_model.wv.index_to_key[:10]):
    print(f"Word: {word}")
    print(f"Vector: {word2vec_model.wv[word]}")

Word: .
Vector: [-2.37311825e-01  6.28960252e-01  3.44021469e-01  3.87754261e-01
  1.58530280e-01 -2.00081065e-01  1.63465798e-01  8.38232458e-01
 -1.58871084e-01 -7.02294931e-02  2.58297265e-01 -4.75914866e-01
  2.84580052e-01  3.13989758e-01  4.65989970e-02 -1.87546745e-01
  2.91182131e-01 -4.16533602e-03 -5.86761832e-01 -5.42694807e-01
  1.67687058e-01  3.71366620e-01  5.87045074e-01 -2.54050255e-01
  2.19895169e-01 -2.54117429e-01 -1.95530020e-02 -2.90364414e-01
 -9.43559110e-02  7.16745481e-02 -4.58040871e-02 -5.99979237e-02
  2.92942077e-01 -4.29699451e-01 -3.60429108e-01  4.27669853e-01
  1.63066626e-01  2.15178668e-01 -2.40467533e-01 -3.62771749e-01
  1.97143272e-01 -4.47333902e-01 -2.22943336e-01  3.11515212e-01
  4.27108258e-01  1.92653999e-01 -3.75076950e-01  1.83154628e-01
  1.68692574e-01  1.33203059e-01  2.82438248e-01 -2.50694811e-01
 -1.60493746e-01 -2.40532443e-01 -3.15279007e-01  3.09041496e-02
  4.82848585e-01 -1.17008708e-01 -1.69959888e-01  1.16920702e-01
 -1.07029